# Bayesian Data Analysis
## PPCA with missing data in STAN

In [3]:
import numpy as np
import pystan
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import linalg
import copy
# edit default plot settings (colours from colorbrewer2.org)
plt.rc('figure', figsize=(8,6))
plt.rc('font', size=14)
plt.rc('lines', color='#377eb8', linewidth=2)
plt.rc('axes', color_cycle=('#377eb8','#e41a1c','#4daf4a',
                            '#984ea3','#ff7f00','#ffff33'))

/Users/nbip/anaconda/lib/python3.5/site-packages/matplotlib/__init__.py:878: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [4]:
class createData:
    'documentation'
    
    def __init__(self,d,dl,n,mu,sigma2):
        # sample latent variable Z
        self.Z = np.random.multivariate_normal(np.zeros(dl),np.identity(dl),n).T  # dl x n    

        # create additive noise
        noise = np.random.multivariate_normal(np.zeros(d),sigma2*np.identity(d),n).T  # d x n
        
        # create orthogonal signal directions
        self.W = linalg.orth(np.random.random((d,dl)))  # d x dl

        # data-space data
        self.Y = np.dot(self.W,self.Z) + noise;
        
        
    def removeData(self,m):
        # copy of original data
        y_miss = copy.copy(self.Y)
        # indicator matrix, 1 for present, 0 for missing
        S = np.ones((d,n))        
        
        # matrix of uniformly distributed random numbers
        r = np.random.uniform(0,1,(d,n))
        
        # boolean of those lower than specified missing rate
        bol = r<m
        
        # set entries to nan as specified by bol
        y_miss[bol] = np.nan
        S[bol] = 0
        
        return dict(y_miss=y_miss,S=S)

In [5]:
d=5
dl=1
n=200
mu=np.zeros(d)
sigma2=0.05

datObj = createData(d,dl,n,mu,sigma2)
# extract quantities
Y = datObj.Y
W = datObj.W
Z = datObj.Z

In [6]:
# missing rate
m = 0.2
missDict = datObj.removeData(m)
y_miss = missDict['y_miss']
S = missDict['S']

## Prepare data for Stan code

In [8]:
dat_complete = np.squeeze(y_miss[S==1])
Ncomp = len(dat_complete) # number of present observations
ind_pres = np.where(S==1)
ind_pres = np.asarray(ind_pres).T
ind_pres = [x+1 for x in ind_pres] # conform to Stan matrix notation
ind_pres = np.asarray(ind_pres)

ind_miss = np.where(S==0)
ind_miss = np.asarray(ind_miss).T
Nmiss = len(ind_miss)
ind_miss = [x+1 for x in ind_miss] # conform to Stan matrix notation
ind_miss = np.asarray(ind_miss)

### Stan model for missing data in PPCA

In [10]:
# =============== PPCA model for missing data ===================
ppca_missing_code = """
data {
  int<lower=1> N;              // num datapoints
  int<lower=1> D;              // num dimension
  int<lower=1> K;              // num basis
  real<lower=0> ones;
 
  int<lower=0> Ncomp; // Number of non-missing values
  int<lower=0> Nmiss; // Number of missing values
 
  real dat_complete[Ncomp];   // Vector of non-missing values
  int ind_pres[Ncomp,2];      // Matrix (row, col) of non-missing value indices
  int ind_miss[Nmiss,2];      // Matrix (row, col) of missing value indices
}
transformed data {
  matrix[K,K] Sigma;
  vector<lower=0>[K] diag_elem;
  vector<lower=0>[K] zr_vec;
  for (k in 1:K) zr_vec[k] <- 0;
  for (k in 1:K) diag_elem[k] <- ones;
  Sigma <- diag_matrix(diag_elem);
}
parameters {
  matrix[D,K] A;            // basis
  vector[K] x[N];           // coefficients
  real<lower=0> sigma;      // noise variance
 
  // Vector containing "stochastic" nodes (for filling in missing values)
  real ymiss[Nmiss]; 
}
transformed parameters{
 
    real y[D,N]; // The "data" with interpolated missing values
 
  // Fill y with non-missing values 
  for(n in 1:Ncomp) 
    y[ind_pres[n,1]][ind_pres[n,2]] <- dat_complete[n];
  
  // Fill the rest of y with missing value "parameters"
  for(n in 1:Nmiss)
    y[ind_miss[n,1]][ind_miss[n,2]] <- ymiss[n];
  
}
model {  
  
  for (i in 1:N)
      x[i] ~ multi_normal(zr_vec, Sigma);
      
  
  for (i in 1:N)
    for (d in 1:D)
      //y[d,i] ~ normal(dot_product(row(A, d), x[i]), sigma);
      increment_log_prob(normal_log(y[d,i], dot_product(row(A, d), x[i]), sigma));      
}
generated quantities {
    vector[N] log_lik;
    for (n in 1:N)
       for (d in 1:D)
        log_lik[n] <- normal_log(y[d,n], dot_product(row(A, d), x[n]), sigma);
}
"""
data = dict(N=n, D=d, K=dl, ones=1, 
            Ncomp=Ncomp, Nmiss=Nmiss, 
            dat_complete=dat_complete, 
            ind_pres=ind_pres, ind_miss = ind_miss)
fit = pystan.stan(model_code=ppca_missing_code, data=data)
print(fit)
samples = fit.extract(permuted=True)



INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_90d0aea0ce0a7e2abb257d75a72c494b NOW.


Inference for Stan model: anon_model_90d0aea0ce0a7e2abb257d75a72c494b.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

               mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
A[0,0]       9.1e-5    0.37   0.53  -0.58  -0.52-9.9e-3   0.52   0.58      2  17.65
A[1,0]      -1.1e-4    0.06   0.09  -0.12  -0.09-3.2e-3   0.09   0.12      2   5.13
A[2,0]       4.2e-4    0.28   0.39  -0.44  -0.39-1.6e-3   0.39   0.44      2  16.09
A[3,0]       1.5e-4    0.45   0.64   -0.7  -0.63-1.2e-3   0.63    0.7      2  18.04
A[4,0]       1.0e-4    0.29    0.4  -0.45   -0.4-1.2e-3    0.4   0.45      2  15.83
x[0,0]       2.3e-4     0.4   0.57  -0.88  -0.52 5.0e-3   0.52   0.89      2   2.67
x[1,0]      -3.3e-3    0.77   1.09  -1.44  -1.07   0.03   1.06   1.44      2   5.25
x[2,0]       7.4e-3    0.06   0.29  -0.57  -0.18 6.7e-3    0.2   0.58     26   1.04
x[3,0]      -9.9e-4    1.22   1.73  -2.12   -1.7   0.09

# Check the weights

In [26]:
W

array([[ 0.53803598],
       [ 0.09152094],
       [ 0.40260148],
       [ 0.61655217],
       [ 0.39989582]])

In [31]:
A = samples['A']
np.mean(abs(A),axis=0)

array([[ 0.52519175],
       [ 0.08691458],
       [ 0.39325943],
       [ 0.63434214],
       [ 0.40345918]])

# Check the noise variance

In [37]:
tmp = samples['sigma']
sigma = [x*x for x in tmp];
s2 = np.mean(sigma)
[s2, sigma2]

[0.049257487064332674, 0.05]